In [1]:
import dotenv
import instagrapi
import httpx

from config import InstaSettings
from services.insta_service import InstaService
from dto.instagram import *

dotenv.load_dotenv()

True

In [2]:
insta_settings = InstaSettings()
client = instagrapi.Client()

In [3]:
client.login(username=insta_settings.USERNAME, password=insta_settings.PASSWORD)

True

In [4]:
url = "https://www.instagram.com/p/DK7G5fVxirh/?utm_source=ig_web_copy_link"
url = InstaService.process_url(url)
url

'https://www.instagram.com/p/DK7G5fVxirh/'

In [5]:
media_pk = client.media_pk_from_url(url)
media_info = MyMedia.model_validate(client.media_info(media_pk), from_attributes=True)

In [6]:
media_info.model_dump()

{'pk': 3655545861481835233,
 'id': '3655545861481835233_73585408380',
 'code': 'DK7G5fVxirh',
 'taken_at': datetime.datetime(2025, 6, 15, 13, 44, 6, tzinfo=TzInfo(UTC)),
 'media_type': <MediaType.IMAGE: 1>,
 'image_versions2': {'candidates': [{'url': HttpUrl('https://scontent-ams4-1.cdninstagram.com/v/t51.2885-15/505493920_17856200562448381_2787234903031138753_n.jpg?stp=dst-jpg_e35_p1080x1080_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6IkZFRUQuaW1hZ2VfdXJsZ2VuLjE0NDB4MTgwMC5zZHIuZjc1NzYxLmRlZmF1bHRfaW1hZ2UifQ&_nc_ht=scontent-ams4-1.cdninstagram.com&_nc_cat=101&_nc_oc=Q6cZ2QEfD0tYdNy5QKF2Kc2cPxfOQbokZzhSEDe8nPcUAtcLMgP6UUkSqW5WUj8DTzkjUcM&_nc_ohc=S1YUde9f0EMQ7kNvwEXlhED&_nc_gid=WB50IC6hVY809KQ8nRynlA&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=MzY1NTU0NTg2MTQ4MTgzNTIzMw%3D%3D.3-ccb7-5&oh=00_AfOTPr-CSE62orgLmZIbNGLVbZVzt4LF_hvsfUB7aUtfKg&oe=6859A37F&_nc_sid=fc8dfb'),
    'height': 1350,
    'width': 1080},
   {'url': HttpUrl('https://scontent-ams4-1.cdninstagram.com/v/t51.2885-15/505493920_17856200562448381_2

In [8]:
file = media_info.extract_media_urls()
file

['https://scontent-ams4-1.cdninstagram.com/v/t51.2885-15/505493920_17856200562448381_2787234903031138753_n.jpg?stp=dst-jpg_e35_p1080x1080_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6IkZFRUQuaW1hZ2VfdXJsZ2VuLjE0NDB4MTgwMC5zZHIuZjc1NzYxLmRlZmF1bHRfaW1hZ2UifQ&_nc_ht=scontent-ams4-1.cdninstagram.com&_nc_cat=101&_nc_oc=Q6cZ2QEfD0tYdNy5QKF2Kc2cPxfOQbokZzhSEDe8nPcUAtcLMgP6UUkSqW5WUj8DTzkjUcM&_nc_ohc=S1YUde9f0EMQ7kNvwEXlhED&_nc_gid=WB50IC6hVY809KQ8nRynlA&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=MzY1NTU0NTg2MTQ4MTgzNTIzMw%3D%3D.3-ccb7-5&oh=00_AfOTPr-CSE62orgLmZIbNGLVbZVzt4LF_hvsfUB7aUtfKg&oe=6859A37F&_nc_sid=fc8dfb']

In [12]:
file[0]

'https://scontent-ams4-1.cdninstagram.com/v/t51.2885-15/505493920_17856200562448381_2787234903031138753_n.jpg?stp=dst-jpg_e35_p1080x1080_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6IkZFRUQuaW1hZ2VfdXJsZ2VuLjE0NDB4MTgwMC5zZHIuZjc1NzYxLmRlZmF1bHRfaW1hZ2UifQ&_nc_ht=scontent-ams4-1.cdninstagram.com&_nc_cat=101&_nc_oc=Q6cZ2QEfD0tYdNy5QKF2Kc2cPxfOQbokZzhSEDe8nPcUAtcLMgP6UUkSqW5WUj8DTzkjUcM&_nc_ohc=S1YUde9f0EMQ7kNvwEXlhED&_nc_gid=WB50IC6hVY809KQ8nRynlA&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=MzY1NTU0NTg2MTQ4MTgzNTIzMw%3D%3D.3-ccb7-5&oh=00_AfOTPr-CSE62orgLmZIbNGLVbZVzt4LF_hvsfUB7aUtfKg&oe=6859A37F&_nc_sid=fc8dfb'

In [10]:
result = 0
for url in media_info.extract_media_urls():
    response = httpx.get(url)
    assert response.is_success
    result += len(response.content)

In [13]:
result / (1024 ** 2)

14.41268539428711